# SPBD Assignment 2

This notebook contains the code developed to implement the propoused solutions to this course assignment

Developed by:
    * Lucas Fischer, nº54659
    * Joana Martins, nº54707
    
    
# IMPORT NOTE:

The implemented code runs locally using the sample dataset.

## Getting the dataset

The dataset can be obtain by running the command bellow, or by uploading the file manualy to the work directory if the file is already present in your machine

# TODO ver como é o link para obter os dados

In [ ]:
!curl -o sorted_data.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv

## Creating result folders

This code removes (if already created) and creates new result folders

In [3]:
!rm -rf spark_rdd_results && mkdir spark_rdd_results
!cd spark_rdd_results && mkdir query3

# Setting up dependencies

The first task we must complete is setting up the right dependencies for our solution

In [2]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark import SparkContext
import traceback
import datetime
from datetime import datetime as dt
import calendar
import time
import math
import operator
import numpy as np
import matplotlib.pyplot as plt

spark = SparkSession.builder.master('local[*]').appName('uberfy').getOrCreate()
sc = spark.sparkContext
filename = "./data/sorted_data.csv" #TODO update this with the file location

## Auxiliary functions

Functions created to help in the development of this assignment

In [4]:
def create_row(line):
    """
        Function that creates a structured tuple representing a row in a RDD

        Params:
            line - A line from the input file

        Rerturns:
            A Strcutured tuple with 14 positions
    """
    #Field - Array_position

    #pickup_dt - 0      fare_amount - 8
    #dropoff_dt - 1     tip_amount - 9
    #trip_time - 2      total_amount - 10
    #trip_distance - 3  pickup_cell - 11
    #pickup_long - 4    dropoff_cell - 12
    #pickup_lat - 5     taxi_id = 13
    #dropoff_long - 6
    #dropoff_lat - 7
    
    splitted_line = line.split(',')
    return (
        splitted_line[2], splitted_line[3], int(splitted_line[4]), float(splitted_line[5]), float(splitted_line[6]), \
        float(splitted_line[7]), float(splitted_line[8]), float(splitted_line[9]), float(splitted_line[11]), \
        float(splitted_line[14]), float(splitted_line[16]), estimate_cellid(float(splitted_line[7]), float(splitted_line[6])),\
        estimate_cellid(float(splitted_line[9]), float(splitted_line[8])), splitted_line[0]
    )


def create_row_df(line):
    """
        Function that creates a Structured Row object representing a Row in a DataFrame

        Params:
            line - A line from the input file

        Returns:
            A Row object representing a row in a Dataframe
    """
    #Field - Array_position

    #pickup_dt - 0      fare_amount - 8
    #dropoff_dt - 1     tip_amount - 9
    #trip_time - 2      total_amount - 10
    #trip_distance - 3  pickup_cell - 11
    #pickup_long - 4    dropoff_cell - 12
    #pickup_lat - 5     taxi_id = 13
    #dropoff_long - 6
    #dropoff_lat - 7
    
    splitted_line = line.split(',')
    return Row(
        pickup_dt = splitted_line[2], dropoff_dt = splitted_line[3], trip_time = int(splitted_line[4]), \
        trip_distance = float(splitted_line[5]), pickup_long = float(splitted_line[6]), pickup_lat = float(splitted_line[7]), \
        dropoff_long = float(splitted_line[8]), dropoff_lat = float(splitted_line[9]), fare_amount = float(splitted_line[11]), \
        tip_amount = float(splitted_line[14]), total_amount = float(splitted_line[16]), pickup_cell = estimate_cellid(float(splitted_line[7]), float(splitted_line[6])), \
        dropoff_cell = estimate_cellid(float(splitted_line[9]), float(splitted_line[8])), taxi_id = splitted_line[0]
        )   


def filter_lines(line):
    """
        Function that filters out empty lines as well as lines that have coordinates as 0.0000 (non relevant points)

        Params:
            line - A line from the input file

        Returns:
            True if the line passed this condition, False otherwise
    """
    splitted_line = line.split(',')

    lon_min = -74.916578
    lon_max = -73.120784
    lat_min = 40.129716
    lat_max = 41.477183

    return (
        len(line) > 0) and \
        (float(splitted_line[6]) != 0) and \
        (float(splitted_line[8]) != 0 and \
        (float(splitted_line[6]) >= lon_min) and \
        (float(splitted_line[6]) <= lon_max) and \
        (float(splitted_line[7]) >= lat_min) and \
        (float(splitted_line[7]) <= lat_max) and \
        (float(splitted_line[8]) >= lon_min) and \
        (float(splitted_line[8]) <= lon_max) and \
        (float(splitted_line[9]) >= lat_min) and \
        (float(splitted_line[9]) <= lat_max)
        )



def estimate_cellid(lat, lon):
    """
        Function that estimates a cell ID given a latitude and longitude based on the coordinates of cell 1.1

        Params:
            lat - Input latitude for which to find the cellID
            lon - Input longitude for which to fin the cellID

        Returns:
            A String such as 'xxx.xxx' representing the ID of the cell
    """
    x0 = -74.913585 #longitude of cell 1.1
    y0 = 41.474937  #latitude of cell 1.1
    s = 500 #500 meters

    delta_x = 0.005986 / 500.0  #Change in longitude coordinates per meter
    delta_y = 0.004491556 /500.0    #Change in latitude coordinates per meter

    cell_x = 1 + math.floor((1/2) + (lon - x0)/(s * delta_x))
    cell_y = 1 + math.floor((1/2) + (y0 - lat)/(s * delta_y))
    
    return f"{cell_x}.{cell_y}"



def create_key_value(structured_tuple):
    """
        Function that from a structured tuple organizes it into a Key-Value formation.
        The key is a tuple containing both the weekday and the hour.
        The value is a dictionary containing only one item, this dictionary is to be merged on the reducer.

        Params:
            structured_tuple - A tuple representing a line of the input file

        Returns:
            A tuple organized into a Key-Value formation
    """

    weekday = convert_to_weekday(structured_tuple[0])
    hour = convert_to_hour(structured_tuple[0])
    route = f"{structured_tuple[11]}-{structured_tuple[12]}"

    return ((weekday, hour), {route: 1})



def custom_reducer(accum, elem):
    """
        Custom function to be used in reduceByKey.
        This function well merge dictionaries counting the number of times each time appears

        Params:
            accum - An accumulator dictionary
            elem - The dictionary of the current iteration

        Returns:
            The accumulator dictionary updated with information obtained by elem
    """

    #store the only existing item inside elem
    key, value = elem.popitem()
    
    if(key in accum): #If accum already has this key, then update its value
        accum[key] += value
    else:   #If accum does not have this key, add it
        accum[key] = value

    return accum



def convert_to_weekday(date):
    """
        Function that converts a date to weekday

        Params:
            date - Unix timestamp formatted date in string form

        Returns:
            A string with the weekday of the input date
    """
    date_obj = dt.strptime(date, '%Y-%m-%d %H:%M:%S')
    return (calendar.day_name[date_obj.weekday()]).lower()



def convert_to_hour(date):
    """
        Function that gets the hour from a date

        Params:
            date - Unix timestamp formatted date in string form

        Returns:
            The hour portion of the input date
    """
    return date[11:13]


# Query 1

In this first query the objective was to obtain an index where it would be possible to check the 10 most frequent taxi routes on any given weekday and hour.

To do so the group first had to map the latitude and longitude coordinates of every event into a 500m x 500m cell in order to represent a small area.

In [5]:
try:
        
    #timestamp to mesure the time taken
    time_before = dt.now()

    #read csv file (change this to the full dataset instead of just the sample)
    raw_data = sc.textFile(filename)

    #Filtering out non empty lines and lines that have a pick up or drop off coordinates as 0
    #Also filtering lines that have coordinates that would be mapped to cells with ID greater than 300 and lower de 1
    #These lines are considerer outliers (stated in http://debs.org/debs-2015-grand-challenge-taxi-trips/)
    non_empty_lines = raw_data.filter(lambda line: filter_lines(line))

    #Shaping the rdd rows
    fields = non_empty_lines.map(lambda line : create_row(line))

    # ((weekday, hour), {route})
    organized_lines = fields.map(lambda line : create_key_value(line))

    #Group all values by its key, reducing them acording to custom_reducer
    grouped = organized_lines.reduceByKey(lambda accum, elem: custom_reducer(accum, elem))

    #Sort descendingly the dictionaries present in the values and take only the first 10 elements
    top_routes = grouped.mapValues(lambda route_dict: sorted(route_dict, key = route_dict.get, reverse = True)[:10])

    #Store the retrieved results
    top_routes.saveAsTextFile("spark_rdd_results/query1")

    for a in top_routes.take(2):
        print(a)

    time_after = dt.now()
    seconds = (time_after - time_before).total_seconds()
    print("Execution time {} seconds".format(seconds))
    # sc.stop()
except:
    traceback.print_exc()
    # sc.stop()

(('tuesday', '10'), ['157.159-155.162', '158.163-155.162', '157.162-155.162', '158.161-155.162', '156.160-155.162', '161.157-155.162', '157.159-175.157', '157.164-155.162', '158.162-155.162', '154.160-154.162'])
(('tuesday', '20'), ['155.162-157.163', '155.162-157.162', '156.161-158.161', '156.161-155.162', '156.166-157.164', '155.166-155.163', '161.157-160.160', '156.161-157.162', '159.160-159.160', '157.160-155.162'])
Execution time 18.368937 seconds


# Query 2

Another question of interesent is "What are the most profitable areas?". In order to obtain a conclusion the group started by calculating the most profitable areas in a given weekday and hour

In [ ]:
try:

    #timestamp to mesure the time taken
    time_before = dt.now()

    # convert_to_weekday_udf = udf(lambda pickup_date: convert_to_weekday(pickup_date), StringType())
    convert_to_weekday_udf = udf(lambda pickup_date: convert_to_weekday(pickup_date), StringType())
    convert_to_hour_udf = udf(lambda pickup_date: pickup_date[11:13], StringType())

    #read csv file (change this to the full dataset instead of just the sample)
    raw_data = sc.textFile(filename)

    #Filtering out non empty lines and lines that have a pick up or drop off coordinates as 0
    non_empty_lines = raw_data.filter(lambda line: filter_lines(line))

    #Shapping the rdd rows
    fields = non_empty_lines.map(lambda line : create_row_df(line))

    #Creating DataFrame
    lines_df = spark.createDataFrame(fields)

    # Get the dropoffs of the last 15 minutes for each cell
    # get the average of the fare
    profit_by_area_15min = lines_df \
        .groupBy(window("dropoff_dt", "900 seconds"), convert_to_weekday_udf("pickup_dt").alias("weekday"), convert_to_hour_udf("pickup_dt").alias("hour"), "pickup_cell") \
        .agg(avg(lines_df.fare_amount + lines_df.tip_amount).alias("median_fare")) \
        .orderBy("median_fare", ascending = False) \
        .select("weekday", "hour", "pickup_cell")


    # empty_taxis = lines_df \
    #     .groupBy(window("dropoff_dt", "900 seconds"), "dropoff_cell") \
    #     .agg(countDistinct("taxi_id").alias("empty_taxis")) \
    #     .select("dropoff_cell", "empty_taxis")

    profit_by_area_15min.show(2)
    
    profit_by_area_15min.rdd.map(lambda row: ((row.weekday, row.hour), row.pickup_cell)).saveAsTextFile("spark_rdd_results/query2")
    
    time_after = dt.now()
    seconds = (time_after - time_before).total_seconds()
    print("Execution time {} seconds".format(seconds))

    # sc.stop()
except:
    traceback.print_exc()
    # sc.stop()


# Query 3

Falar aqui sobre a query 3

In [6]:
schema = StructType([
    StructField("medallion", StringType()),
    StructField("hack_license", StringType()),
    StructField("pickup_datetime", StringType()),
    StructField("dropoff_datetime", StringType()),
    StructField("trip_time_in_secs", IntegerType()),
    StructField("trip_distance", FloatType()),
    StructField("pickup_longitude", FloatType()),
    StructField("pickup_latitude", FloatType()),
    StructField("dropoff_longitude", FloatType()),
    StructField("dropoff_latitude", FloatType()),
    StructField("payment_type", StringType()),
    StructField("fare_amount", FloatType()),
    StructField("surcharge", FloatType()),
    StructField("mta_tax", FloatType()),
    StructField("tip_amount", FloatType()),
    StructField("tolls_amount", FloatType()),
    StructField("total_amount", FloatType())
])

# Load and parse the data
data = spark.read.schema(schema).option("header", "false").csv("./data/sorted_data.csv")

#Limits to the longitudes and latitudes, everypoint that isn't between these coordinates is considered an outlier
lon_min = -74.916578
lon_max = -73.120784
lat_min = 40.129716
lat_max = 41.477183

filter_data = data.filter(
    (data.pickup_longitude != 0) & \
    (data.pickup_latitude != 0) & \
    (data.dropoff_longitude != 0) & \
    (data.dropoff_latitude != 0) & \
    (data.pickup_longitude <= lon_max) & \
    (data.pickup_longitude >= lon_min) & \
    (data.pickup_latitude >= lat_min) & \
    (data.pickup_latitude <= lat_max) & \
    (data.dropoff_longitude <= lon_max) & \
    (data.dropoff_longitude >= lon_min) & \
    (data.dropoff_latitude >= lat_min) & \
    (data.dropoff_latitude <= lat_max)
    )


#Define the target columns and output column
assembler = VectorAssembler(
    inputCols = ["pickup_latitude", "pickup_longitude"],
    outputCol = "features"
    )

#Transform the data according to the Assembler created above
data_prepared = assembler.transform(filter_data)

for i in [5, 31, 75]: #find other k values

    centroids_file = "spark_rdd_results/query3/centroids_"+str(i)+".txt"
    # Write results to file
    f = open(centroids_file,"w+")

    #Instanciate Kmeans class with the given K value
    kmeans = KMeans(k = i)

    #Fit the data
    model = kmeans.fit(data_prepared)

    #Evalute clustering by computing Sum of Square Errors
    sum_square_error = model.computeCost(data_prepared)

    # To get the prototypes
    centers = model.clusterCenters()
    print("Cluster Centers: ")
    for center in centers:
        f.write('{:.8f}{}{:.8f}{}'.format(center[0],",",center[1],"\n"))

    # Close file where centroid positions are stored
    f.close()

    #The lower the Sum of Square errors is it means the closer the points are to the prototypes, this is equivalent to
    #the sum of the square that each person has to walk to nearest taxi stand, so minimizing it would be optimal
    print(f"{i} -> SSE: {sum_square_error}")

Cluster Centers: 
[ 40.72606307 -73.99603147]
[ 40.78036605 -73.96005749]
[ 40.64674545 -73.78507649]
[ 40.76848466 -73.87468834]
[ 40.75604783 -73.98210939]
5 -> SSE: 607.6047854043499
Cluster Centers: 
[ 40.74337146 -73.98152608]
[ 40.77235547 -73.86963166]
[ 40.64553303 -73.78437704]
[ 40.80018064 -73.94307785]
[ 40.71168096 -74.01065348]
[ 40.80106031 -73.9652392 ]
[ 40.7239783  -74.00040032]
[ 40.76015136 -73.98518381]
[ 40.73898286 -74.00365672]
[ 40.76093345 -73.96967685]
[ 40.62374858 -73.97757702]
[ 40.76927986 -73.95949391]
[ 40.69030608 -73.99218571]
[ 40.78541415 -73.97591658]
[ 40.77984761 -73.95304893]
[ 41.1581104  -74.72800269]
[ 40.70848852 -73.79849706]
[ 40.73882672 -73.99134331]
[ 40.75295661 -73.97573484]
[ 40.67905306 -73.97448264]
[ 40.71432898 -73.95362458]
[ 40.8392866  -73.93688329]
[ 40.73879219 -74.54635698]
[ 41.09031822 -73.84855259]
[ 40.64701287 -74.17645513]
[ 40.75254434 -73.99383228]
[ 40.75578549 -73.92308828]
[ 40.77180636 -73.98355095]
[ 40.7250465